In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, gradient_boosting
pd.options.display.max_columns = 500

In [2]:
train = pd.read_csv("./Data/train.csv")

/Users/ipm/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (294,299,300,302,307,313,315,365,367,369) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
test = pd.read_csv("./Data/test.csv")

/Users/ipm/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (294,299,300,302,313,315,365,367,369) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
print(train.shape)
train.head()

(300000, 377)


,UCIC_ID,NO_OF_Accs,HNW_CATEGORY,vintage,EMAIL_UNSUBSCRIBE,OCCUP_ALL_NEW,city,dependents,zip,FINAL_WORTH_prev1,ENGAGEMENT_TAG_prev1,C_prev1,D_prev1,ATM_C_prev1,ATM_D_prev1,BRANCH_C_prev1,BRANCH_D_prev1,IB_C_prev1,IB_D_prev1,MB_C_prev1,MB_D_prev1,POS_C_prev1,POS_D_prev1,count_C_prev1,count_D_prev1,COUNT_ATM_C_prev1,COUNT_ATM_D_prev1,COUNT_BRANCH_C_prev1,COUNT_BRANCH_D_prev1,COUNT_IB_C_prev1,COUNT_IB_D_prev1,COUNT_MB_C_prev1,COUNT_MB_D_prev1,COUNT_POS_C_prev1,COUNT_POS_D_prev1,custinit_CR_amt_prev1,custinit_DR_amt_prev1,custinit_CR_cnt_prev1,custinit_DR_cnt_prev1,ATM_amt_prev1,ATM_CW_Amt_prev1,ATM_CW_Cnt_prev1,BRN_CW_Amt_prev1,BRN_CW_Cnt_prev1,BRN_CASH_Dep_Amt_prev1,BRN_CASH_Dep_Cnt_prev1,CNR_prev1,BAL_prev1,EOP_prev1,CR_AMB_Prev1,C_prev2,D_prev2,ATM_C_prev2,ATM_D_prev2,BRANCH_C_prev2,BRANCH_D_prev2,IB_C_prev2,IB_D_prev2,MB_C_prev2,MB_D_prev2,POS_C_prev2,POS_D_prev2,count_C_prev2,count_D_prev2,COUNT_ATM_C_prev2,COUNT_ATM_D_prev2,COUNT_BRANCH_C_prev2,COUNT_BRANCH_D_prev2,COUNT_IB_C_prev2,COUNT_IB_D_prev2,COUNT_MB_C_prev2,COUNT_MB_D_prev2,COUNT_POS_C_prev2,COUNT_POS_D_prev2,custinit_CR_amt_prev2,custinit_DR_amt_prev2,custinit_CR_cnt_prev2,custinit_DR_cnt_prev2,ATM_amt_prev2,ATM_CW_Amt_prev2,ATM_CW_Cnt_prev2,BRN_CW_Amt_prev2,BRN_CW_Cnt_prev2,BRN_CASH_Dep_Amt_prev2,BRN_CASH_Dep_Cnt_prev2,CNR_prev2,BAL_prev2,EOP_prev2,CR_AMB_Prev2,C_prev3,D_prev3,ATM_C_prev3,ATM_D_prev3,BRANCH_C_prev3,BRANCH_D_prev3,IB_C_prev3,IB_D_prev3,MB_C_prev3,MB_D_prev3,POS_C_prev3,POS_D_prev3,count_C_prev3,count_D_prev3,COUNT_ATM_C_prev3,COUNT_ATM_D_prev3,COUNT_BRANCH_C_prev3,COUNT_BRANCH_D_prev3,COUNT_IB_C_prev3,COUNT_IB_D_prev3,COUNT_MB_C_prev3,COUNT_MB_D_prev3,COUNT_POS_C_prev3,COUNT_POS_D_prev3,custinit_CR_amt_prev3,custinit_DR_amt_prev3,custinit_CR_cnt_prev3,custinit_DR_cnt_prev3,ATM_amt_prev3,ATM_CW_Amt_prev3,ATM_CW_Cnt_prev3,BRN_CW_Amt_prev3,BRN_CW_Cnt_prev3,BRN_CASH_Dep_Amt_prev3,BRN_CASH_Dep_Cnt_prev3,CNR_prev3,BAL_prev3,EOP_prev3,CR_AMB_Prev3,C_prev4,D_prev4,ATM_C_prev4,ATM_D_prev4,BRANCH_C_prev4,BRANCH_D_prev4,IB_C_prev4,IB_D_prev4,MB_C_prev4,MB_D_prev4,POS_C_prev4,POS_D_prev4,count_C_prev4,count_D_prev4,COUNT_ATM_C_prev4,COUNT_ATM_D_prev4,COUNT_BRANCH_C_prev4,COUNT_BRANCH_D_prev4,COUNT_IB_C_prev4,COUNT_IB_D_prev4,COUNT_MB_C_prev4,COUNT_MB_D_prev4,COUNT_POS_C_prev4,COUNT_POS_D_prev4,custinit_CR_amt_prev4,custinit_DR_amt_prev4,custinit_CR_cnt_prev4,custinit_DR_cnt_prev4,ATM_amt_prev4,ATM_CW_Amt_prev4,ATM_CW_Cnt_prev4,BRN_CW_Amt_prev4,BRN_CW_Cnt_prev4,BRN_CASH_Dep_Amt_prev4,BRN_CASH_Dep_Cnt_prev4,CNR_prev4,BAL_prev4,EOP_prev4,CR_AMB_Prev4,C_prev5,D_prev5,ATM_C_prev5,ATM_D_prev5,BRANCH_C_prev5,BRANCH_D_prev5,IB_C_prev5,IB_D_prev5,MB_C_prev5,MB_D_prev5,POS_C_prev5,POS_D_prev5,count_C_prev5,count_D_prev5,COUNT_ATM_C_prev5,COUNT_ATM_D_prev5,COUNT_BRANCH_C_prev5,COUNT_BRANCH_D_prev5,COUNT_IB_C_prev5,COUNT_IB_D_prev5,COUNT_MB_C_prev5,COUNT_MB_D_prev5,COUNT_POS_C_prev5,COUNT_POS_D_prev5,custinit_CR_amt_prev5,custinit_DR_amt_prev5,custinit_CR_cnt_prev5,custinit_DR_cnt_prev5,ATM_amt_prev5,ATM_CW_Amt_prev5,ATM_CW_Cnt_prev5,BRN_CW_Amt_prev5,BRN_CW_Cnt_prev5,BRN_CASH_Dep_Amt_prev5,BRN_CASH_Dep_Cnt_prev5,CNR_prev5,BAL_prev5,EOP_prev5,CR_AMB_Prev5,C_prev6,D_prev6,ATM_C_prev6,ATM_D_prev6,BRANCH_C_prev6,BRANCH_D_prev6,IB_C_prev6,IB_D_prev6,MB_C_prev6,MB_D_prev6,POS_C_prev6,POS_D_prev6,count_C_prev6,count_D_prev6,COUNT_ATM_C_prev6,COUNT_ATM_D_prev6,COUNT_BRANCH_C_prev6,COUNT_BRANCH_D_prev6,COUNT_IB_C_prev6,COUNT_IB_D_prev6,COUNT_MB_C_prev6,COUNT_MB_D_prev6,COUNT_POS_C_prev6,COUNT_POS_D_prev6,custinit_CR_amt_prev6,custinit_DR_amt_prev6,custinit_CR_cnt_prev6,custinit_DR_cnt_prev6,ATM_amt_prev6,ATM_CW_Amt_prev6,ATM_CW_Cnt_prev6,BRN_CW_Amt_prev6,BRN_CW_Cnt_prev6,BRN_CASH_Dep_Amt_prev6,BRN_CASH_Dep_Cnt_prev6,CNR_prev6,BAL_prev6,EOP_prev6,CR_AMB_Prev6,FRX_PrevQ1,EFT_SELF_TRANSFER_PrevQ1,Billpay_Active_PrevQ1,Billpay_Reg_ason_Prev1,FD_AMOUNT_BOOK_PrevQ1,FD_AMOUNT_BOOK_PrevQ2,NO_OF_FD_BOOK_PrevQ1,NO_OF_FD_BOOK_PrevQ2,NO_OF_RD_BOOK_PrevQ1,NO_OF_RD_BOOK_PrevQ2,RD_AMOUNT_BOOK_Pr

In [5]:
train['Responders'].value_counts() / train.shape[0]

0    0.827677
1    0.172323
Name: Responders, dtype: float64

In [6]:
train.describe()

/Users/ipm/anaconda/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,UCIC_ID,NO_OF_Accs,vintage,dependents,zip,C_prev1,D_prev1,ATM_C_prev1,ATM_D_prev1,BRANCH_C_prev1,BRANCH_D_prev1,IB_C_prev1,IB_D_prev1,MB_C_prev1,MB_D_prev1,POS_C_prev1,POS_D_prev1,count_C_prev1,count_D_prev1,COUNT_ATM_C_prev1,COUNT_ATM_D_prev1,COUNT_BRANCH_C_prev1,COUNT_BRANCH_D_prev1,COUNT_IB_C_prev1,COUNT_IB_D_prev1,COUNT_MB_C_prev1,COUNT_MB_D_prev1,COUNT_POS_C_prev1,COUNT_POS_D_prev1,custinit_CR_amt_prev1,custinit_DR_amt_prev1,custinit_CR_cnt_prev1,custinit_DR_cnt_prev1,ATM_amt_prev1,ATM_CW_Amt_prev1,ATM_CW_Cnt_prev1,BRN_CW_Amt_prev1,BRN_CW_Cnt_prev1,BRN_CASH_Dep_Amt_prev1,BRN_CASH_Dep_Cnt_prev1,CNR_prev1,BAL_prev1,EOP_prev1,CR_AMB_Prev1,C_prev2,D_prev2,ATM_C_prev2,ATM_D_prev2,BRANCH_C_prev2,BRANCH_D_prev2,IB_C_prev2,IB_D_prev2,MB_C_prev2,MB_D_prev2,POS_C_prev2,POS_D_prev2,count_C_prev2,count_D_prev2,COUNT_ATM_C_prev2,COUNT_ATM_D_prev2,COUNT_BRANCH_C_prev2,COUNT_BRANCH_D_prev2,COUNT_IB_C_prev2,COUNT_IB_D_prev2,COUNT_MB_C_prev2,COUNT_MB_D_prev2,COUNT_POS_C_prev2,COUNT_POS_D_prev2,custinit_CR_amt_prev2,custinit_DR_amt_prev2,custinit_CR_cnt_prev2,custinit_DR_cnt_prev2,ATM_amt_prev2,ATM_CW_Amt_prev2,ATM_CW_Cnt_prev2,BRN_CW_Amt_prev2,BRN_CW_Cnt_prev2,BRN_CASH_Dep_Amt_prev2,BRN_CASH_Dep_Cnt_prev2,CNR_prev2,BAL_prev2,EOP_prev2,CR_AMB_Prev2,C_prev3,D_prev3,ATM_C_prev3,ATM_D_prev3,BRANCH_C_prev3,BRANCH_D_prev3,IB_C_prev3,IB_D_prev3,MB_C_prev3,MB_D_prev3,POS_C_prev3,POS_D_prev3,count_C_prev3,count_D_prev3,COUNT_ATM_C_prev3,COUNT_ATM_D_prev3,COUNT_BRANCH_C_prev3,COUNT_BRANCH_D_prev3,COUNT_IB_C_prev3,COUNT_IB_D_prev3,COUNT_MB_C_prev3,COUNT_MB_D_prev3,COUNT_POS_C_prev3,COUNT_POS_D_prev3,custinit_CR_amt_prev3,custinit_DR_amt_prev3,custinit_CR_cnt_prev3,custinit_DR_cnt_prev3,ATM_amt_prev3,ATM_CW_Amt_prev3,ATM_CW_Cnt_prev3,BRN_CW_Amt_prev3,BRN_CW_Cnt_prev3,BRN_CASH_Dep_Amt_prev3,BRN_CASH_Dep_Cnt_prev3,CNR_prev3,BAL_prev3,EOP_prev3,CR_AMB_Prev3,C_prev4,D_prev4,ATM_C_prev4,ATM_D_prev4,BRANCH_C_prev4,BRANCH_D_prev4,IB_C_prev4,IB_D_prev4,MB_C_prev4,MB_D_prev4,POS_C_prev4,POS_D_prev4,count_C_prev4,count_D_prev4,COUNT_ATM_C_prev4,COUNT_ATM_D_prev4,COUNT_BRANCH_C_prev4,COUNT_BRANCH_D_prev4,COUNT_IB_C_prev4,COUNT_IB_D_prev4,COUNT_MB_C_prev4,COUNT_MB_D_prev4,COUNT_POS_C_prev4,COUNT_POS_D_prev4,custinit_CR_amt_prev4,custinit_DR_amt_prev4,custinit_CR_cnt_prev4,custinit_DR_cnt_prev4,ATM_amt_prev4,ATM_CW_Amt_prev4,ATM_CW_Cnt_prev4,BRN_CW_Amt_prev4,BRN_CW_Cnt_prev4,BRN_CASH_Dep_Amt_prev4,BRN_CASH_Dep_Cnt_prev4,CNR_prev4,BAL_prev4,EOP_prev4,CR_AMB_Prev4,C_prev5,D_prev5,ATM_C_prev5,ATM_D_prev5,BRANCH_C_prev5,BRANCH_D_prev5,IB_C_prev5,IB_D_prev5,MB_C_prev5,MB_D_prev5,POS_C_prev5,POS_D_prev5,count_C_prev5,count_D_prev5,COUNT_ATM_C_prev5,COUNT_ATM_D_prev5,COUNT_BRANCH_C_prev5,COUNT_BRANCH_D_prev5,COUNT_IB_C_prev5,COUNT_IB_D_prev5,COUNT_MB_C_prev5,COUNT_MB_D_prev5,COUNT_POS_C_prev5,COUNT_POS_D_prev5,custinit_CR_amt_prev5,custinit_DR_amt_prev5,custinit_CR_cnt_prev5,custinit_DR_cnt_prev5,ATM_amt_prev5,ATM_CW_Amt_prev5,ATM_CW_Cnt_prev5,BRN_CW_Amt_prev5,BRN_CW_Cnt_prev5,BRN_CASH_Dep_Amt_prev5,BRN_CASH_Dep_Cnt_prev5,CNR_prev5,BAL_prev5,EOP_prev5,CR_AMB_Prev5,C_prev6,D_prev6,ATM_C_prev6,ATM_D_prev6,BRANCH_C_prev6,BRANCH_D_prev6,IB_C_prev6,IB_D_prev6,MB_C_prev6,MB_D_prev6,POS_C_prev6,POS_D_prev6,count_C_prev6,count_D_prev6,COUNT_ATM_C_prev6,COUNT_ATM_D_prev6,COUNT_BRANCH_C_prev6,COUNT_BRANCH_D_prev6,COUNT_IB_C_prev6,COUNT_IB_D_prev6,COUNT_MB_C_prev6,COUNT_MB_D_prev6,COUNT_POS_C_prev6,COUNT_POS_D_prev6,custinit_CR_amt_prev6,custinit_DR_amt_prev6,custinit_CR_cnt_prev6,custinit_DR_cnt_prev6,ATM_amt_prev6,ATM_CW_Amt_prev6,ATM_CW_Cnt_prev6,BRN_CW_Amt_prev6,BRN_CW_Cnt_prev6,BRN_CASH_Dep_Amt_prev6,BRN_CASH_Dep_Cnt_prev6,CNR_prev6,BAL_prev6,EOP_prev6,CR_AMB_Prev6,FRX_PrevQ1,Billpay_Active_PrevQ1,Billpay_Reg_ason_Prev1,FD_AMOUNT_BOOK_PrevQ1,FD_AMOUNT_BOOK_PrevQ2,NO_OF_FD_BOOK_PrevQ1,NO_OF_FD_BOOK_PrevQ2,NO_OF_RD_BOOK_PrevQ1,NO_OF_RD_BOOK_PrevQ2,RD_AMOUNT_BOOK_PrevQ1,RD_AMOUNT_BOOK_PrevQ2,Total_Invest_in_MF_PrevQ1,Total_Invest_in_MF_PrevQ2,count_No_of_MF_PrevQ1,count_No_of_M

In [7]:
train['type'] = 'Train'
test['type'] = 'Test'

In [8]:
comb = pd.concat([train.drop(['Responders'], axis=1), test])
comb.shape

(500000, 377)

In [9]:
np.set_printoptions()
comb.columns[comb.isnull().any()].values.tolist()

['EMAIL_UNSUBSCRIBE',
 'OCCUP_ALL_NEW',
 'city',
 'dependents',
 'zip',
 'FINAL_WORTH_prev1',
 'ENGAGEMENT_TAG_prev1',
 'custinit_CR_amt_prev1',
 'custinit_DR_amt_prev1',
 'custinit_CR_cnt_prev1',
 'custinit_DR_cnt_prev1',
 'ATM_amt_prev1',
 'ATM_CW_Amt_prev1',
 'ATM_CW_Cnt_prev1',
 'BRN_CW_Amt_prev1',
 'BRN_CW_Cnt_prev1',
 'BRN_CASH_Dep_Amt_prev1',
 'BRN_CASH_Dep_Cnt_prev1',
 'custinit_CR_amt_prev2',
 'custinit_DR_amt_prev2',
 'custinit_CR_cnt_prev2',
 'custinit_DR_cnt_prev2',
 'ATM_amt_prev2',
 'ATM_CW_Amt_prev2',
 'ATM_CW_Cnt_prev2',
 'BRN_CW_Amt_prev2',
 'BRN_CW_Cnt_prev2',
 'BRN_CASH_Dep_Amt_prev2',
 'BRN_CASH_Dep_Cnt_prev2',
 'custinit_CR_amt_prev3',
 'custinit_DR_amt_prev3',
 'custinit_CR_cnt_prev3',
 'custinit_DR_cnt_prev3',
 'ATM_amt_prev3',
 'ATM_CW_Amt_prev3',
 'ATM_CW_Cnt_prev3',
 'BRN_CW_Amt_prev3',
 'BRN_CW_Cnt_prev3',
 'BRN_CASH_Dep_Amt_prev3',
 'BRN_CASH_Dep_Cnt_prev3',
 'custinit_CR_amt_prev4',
 'custinit_DR_amt_prev4',
 'custinit_CR_cnt_prev4',
 'custinit_DR_cnt_prev4

In [10]:
cat_col = []
num_col = []
for col in comb.columns:
    if comb[col].dtype == 'object':
        cat_col.append(col)
    elif comb[col].dtype != 'object':
        num_col.append(col)
print len(cat_col)
print len(num_col)

39
338


In [11]:
na_n_cols = ['EMAIL_UNSUBSCRIBE', 'AGRI_TAG_LIVE',
 'AL_CNC_TAG_LIVE',
 'AL_TAG_LIVE',
 'BL_TAG_LIVE',
 'CC_TAG_LIVE',
 'CE_TAG_LIVE',
 'CV_TAG_LIVE',
 'DEMAT_TAG_LIVE',
 'EDU_TAG_LIVE',
 'GL_TAG_LIVE',
 'HL_TAG_LIVE',
 'SEC_ACC_TAG_LIVE',
 'INS_TAG_LIVE',
 'LAS_TAG_LIVE',
 'MF_TAG_LIVE',
 'OTHER_LOANS_TAG_LIVE',
 'PL_TAG_LIVE',
 'RD_TAG_LIVE',
 'FD_TAG_LIVE',
 'TL_TAG_LIVE',
 'TWL_TAG_LIVE',
 'lap_tag_live']
for col in comb.columns:
    if col in na_n_cols and comb[col].isnull().sum > 0:
        comb[col].fillna('N', inplace=True)

In [12]:
for col in cat_col:
    print comb[col].value_counts()

2_Preferred    257036
3_Classic      179158
1_Imperia       63806
Name: HNW_CATEGORY, dtype: int64
N    476702
Y     23298
Name: EMAIL_UNSUBSCRIBE, dtype: int64
SELF_EMPLOYED    166791
SALARIED         117525
HOUSEWIFE         95780
INDIVIDUAL        46173
STUDENT           36461
RETIRED           35128
MISSING            1243
NON_INDIVIDUA       745
Name: OCCUP_ALL_NEW, dtype: int64
MUMBAI       60906
NEW DELHI    34256
DELHI        23890
BANGALORE    22052
KOLKATA      20054
CHENNAI      16722
HYDERABAD    14597
PUNE         14398
AHMEDABAD    11908
GURGAON      11488
LUDHIANA      6413
THANE         6347
NOIDA         5356
X             5252
JAIPUR        5075
SURAT         4680
FARIDABAD     4651
CHANDIGAR     4515
GHAZIABAD     4259
NAVI MUMB     4214
VADODARA      4073
AMRITSAR      3909
LUCKNOW       3708
JALANDHAR     3631
RAJKOT        2851
INDORE        2663
PATNA         2339
MOHALI        2300
KANPUR        2254
THRISSUR      2083
             ...  
AMTA             1
VASAN

In [13]:
comb['Complaint_Resolved_PrevQ1'].fillna(99, inplace=True)
comb['Query_Resolved_PrevQ1'].fillna(99, inplace=True)
comb['Req_Resolved_PrevQ1'].fillna(99, inplace=True)
comb['OCCUP_ALL_NEW'].fillna('MISSING', inplace=True)
comb['FINAL_WORTH_prev1'].fillna('NO', inplace=True)
comb['ENGAGEMENT_TAG_prev1'].fillna('N', inplace=True)

In [14]:
for col in num_col:
    print col, comb[col].isnull().sum()

UCIC_ID 0
NO_OF_Accs 0
vintage 0
dependents 43829
zip 1768
C_prev1 0
D_prev1 0
ATM_C_prev1 0
ATM_D_prev1 0
BRANCH_C_prev1 0
BRANCH_D_prev1 0
IB_C_prev1 0
IB_D_prev1 0
MB_C_prev1 0
MB_D_prev1 0
POS_C_prev1 0
POS_D_prev1 0
count_C_prev1 0
count_D_prev1 0
COUNT_ATM_C_prev1 0
COUNT_ATM_D_prev1 0
COUNT_BRANCH_C_prev1 0
COUNT_BRANCH_D_prev1 0
COUNT_IB_C_prev1 0
COUNT_IB_D_prev1 0
COUNT_MB_C_prev1 0
COUNT_MB_D_prev1 0
COUNT_POS_C_prev1 0
COUNT_POS_D_prev1 0
custinit_CR_amt_prev1 359094
custinit_DR_amt_prev1 193260
custinit_CR_cnt_prev1 359094
custinit_DR_cnt_prev1 193260
ATM_amt_prev1 383730
ATM_CW_Amt_prev1 392191
ATM_CW_Cnt_prev1 392191
BRN_CW_Amt_prev1 499509
BRN_CW_Cnt_prev1 499509
BRN_CASH_Dep_Amt_prev1 443900
BRN_CASH_Dep_Cnt_prev1 443900
CNR_prev1 0
BAL_prev1 0
EOP_prev1 0
CR_AMB_Prev1 0
C_prev2 0
D_prev2 0
ATM_C_prev2 0
ATM_D_prev2 0
BRANCH_C_prev2 0
BRANCH_D_prev2 0
IB_C_prev2 0
IB_D_prev2 0
MB_C_prev2 0
MB_D_prev2 0
POS_C_prev2 0
POS_D_prev2 0
count_C_prev2 0
count_D_prev2 0
COUNT_A

In [15]:
comb = comb.fillna(0)

In [16]:
comb.loc[comb['Req_Resolved_PrevQ1'] == '>', 'Req_Resolved_PrevQ1'] = '10'
comb.loc[comb['Query_Resolved_PrevQ1'] == '>', 'Query_Resolved_PrevQ1'] = '10'
comb.loc[comb['Complaint_Resolved_PrevQ1'] == '>', 'Complaint_Resolved_PrevQ1'] = '10'

In [17]:
comb['Req_Resolved_PrevQ1'] = pd.to_numeric(comb['Req_Resolved_PrevQ1'], errors='coerce')
comb['Query_Resolved_PrevQ1'] = pd.to_numeric(comb['Query_Resolved_PrevQ1'], errors='coerce')
comb['Complaint_Resolved_PrevQ1'] = pd.to_numeric(comb['Complaint_Resolved_PrevQ1'], errors='coerce')

In [25]:
comb1 = comb.drop('city', axis=1)

In [ ]:
print cat_col.remove('city')

In [37]:
from sklearn.preprocessing import LabelEncoder
for var in cat_col:
    le = LabelEncoder()
    comb1[var] = le.fit_transform(comb1[var].astype('str'))

In [38]:
comb1.shape

(500000, 376)

In [40]:
 comb1['type'].value_counts()

1    300000
0    200000
Name: type, dtype: int64

In [41]:
train1 = comb1.loc[comb1['type'] == 1]
test1 = comb1.loc[comb1['type'] == 0]

In [44]:
X = train1.drop(['UCIC_ID'], axis=1)
y = train['Responders']
rf = RandomForestClassifier()
rf.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [50]:
test1.drop(['type'], axis=1, inplace=True)
rf.predict(test1)

/Users/ipm/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


array([0, 1, 0, ..., 0, 0, 1])

In [51]:
from collections import Counter
Counter(rf.predict(test1))

Counter({0: 128297, 1: 71703})